<a href="https://colab.research.google.com/github/enverakbacak/EMIR/blob/main/myGenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pwd

/content


In [3]:
cd drive/MyDrive/ColabNotebooks/generators/

/content/drive/MyDrive/ColabNotebooks/generators


In [4]:
#coding=utf-8
import tensorflow as tf
import cv2
import glob
import numpy as np
import sys,os
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Agg')
import time
import glob
import os, os.path
import re
import scipy.io
from sklearn.model_selection import train_test_split
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import Input
import tensorflow.keras.optimizers
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import InputLayer
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import model_from_json
from tensorflow.keras.models import load_model
import pandas as pd  
import matplotlib.pyplot as plt
from skimage.transform import resize   # for resizing images
from tensorflow.keras.optimizers import SGD
import math

In [5]:
def generator(samples, labels, batch_size=16):
    """
    Yields the next training batch.
    Suppose `samples` is an array [[image1_filename,label1], [image2_filename,label2],...].
    """
    num_samples = len(samples)

    while True: # Loop forever so the generator never terminates

        # Get index to start each batch: [0, batch_size, 2*batch_size, ..., max multiple of batch_size &lt;= num_samples]
        for offset in range(0, num_samples, batch_size):
            # Get the samples you'll use in this batch
            batch_samples = samples[offset:offset+batch_size]
            label_samples = labels[offset:offset+batch_size]

            # Initialise X_train and y_train arrays for this batch
            X_train = []
            y_train = []
            # For each example
            for batch_sample in batch_samples:
              #filename = batch_sample
              image = plt.imread(batch_sample[0])
              image = cv2.resize(image,(229, 229))
              X_train.append(image)
            X_train = np.array(X_train)
            X_train = X_train.astype(float)
            #print(X_train.dtype)
            #from tensorflow.keras.applications.vgg16 import preprocess_input
            #X_train = preprocess_input(X_train) # preprocessing the input data

            for label_sample in label_samples:
              y_train.append(label_sample)
            y_train = np.array(y_train)
            y_train = y_train.astype(float)
            #print(y_train.dtype)

            # The generator-y part: yield the next training batch
            yield X_train, y_train


# Import list of train and validation data (image filenames and image labels)
# Note this is not valid code.


In [6]:
data=[]
labels=[]
from csv import reader
# skip first line i.e. read header first and then iterate over each row od csv as a list
with open('data.csv', 'r') as read_obj:
    csv_reader = reader(read_obj)
    header = next(csv_reader)
    # Check file as empty
    if header != None:
        # Iterate over each row after the header in the csv
        for row in csv_reader:
            # row variable is a list that represents a row in csv
            data.append(row)
data=np.array(data)
print(data.dtype)
print(data.shape)



# skip first line i.e. read header first and then iterate over each row od csv as a list
with open('Y.csv', 'r') as read_obj:
    csv_reader = reader(read_obj)
    header = next(csv_reader)
    # Check file as empty
    if header != None:
        # Iterate over each row after the header in the csv
        for row in csv_reader:
            # row variable is a list that represents a row in csv
            labels.append(row)
labels=np.array(labels)
print(labels.shape)
print(labels.dtype)

#labels = np.asarray(labels).astype('float32').reshape((-1,1))
#print(labels.shape[:])

<U21
(2000, 1)
(2000, 5)
<U1


In [7]:
# Create generator
train_generator = generator(data, labels, batch_size=32)
x,y = next(train_generator)

In [8]:
print ('x_shape: ', x.shape)
print ('labels_shape: ', y.shape)

x_shape:  (32, 229, 229, 3)
labels_shape:  (32, 5)


In [9]:
x,y = next(train_generator)

In [10]:
print(y[:])

[[1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 1. 1.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 1. 0.]
 [1. 0. 0. 1. 0.]
 [1. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 1.]
 [1. 0. 0. 1. 0.]
 [1. 0. 0. 0. 1.]
 [1. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0.]]


In [11]:
image_size = 229
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))
base_model.trainable = False

batch_size = 16
hash_bits = 64
inputs    = tensorflow.keras.Input(shape = (image_size, image_size,3))
x         = base_model(inputs)
flatten   = Flatten()(x)
Dense_1   = Dense(1024)(flatten)
#avgPool = GlobalAveragePooling2D()(Dense_1)
Dense_2 = Dense(hash_bits ,activation='sigmoid')(Dense_1)
Dense_3 = Dense(5, activation='sigmoid')(Dense_2)
model = Model(inputs, Dense_3)
print(model.summary())



sgd = SGD(learning_rate=0.0001, decay=1e-6, momentum=0.9, nesterov=True)
def c_loss_1(y_true, y_pred):
    return  tf.keras.losses.binary_crossentropy(y_true, y_pred)

def c_loss_2(noise_1, noise_2):
    #noise_1 = tf.cast(Dense_2 > 0.5 , tf.float32 )
    noise_1 = (Dense_2 > 0.5 )
    noise_2 = Dense_2
    return  tf.keras.losses.binary_crossentropy(noise_1, noise_2)

model.compile(loss = [c_loss_1, c_loss_2],  optimizer=sgd, metrics=['accuracy'],)


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 229, 229, 3)]     0         
_________________________________________________________________
inception_v3 (Functional)    (None, 5, 5, 2048)        21802784  
_________________________________________________________________
flatten (Flatten)            (None, 51200)             0         
_________________________________________________________________
dense (Dense)                (None, 1024)              52429824  
_________________________________________________________________
dense_1 (Dense)              (None, 64)                65600     
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 325       
Total params: 74,298,533
Trainable params: 52,495,749
Non-trainable params: 21,802,784
________________________________________

In [ ]:
model.fit_generator(train_generator, steps_per_epoch = math.ceil(len(labels) // batch_size), verbose=1, epochs=300)

Epoch 1/300


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


125/125 [==============================] - 21s 120ms/step - loss: 0.6292 - accuracy: 0.1603
Epoch 2/300
125/125 [==============================] - 15s 119ms/step - loss: 0.5787 - accuracy: 0.2868
Epoch 3/300
125/125 [==============================] - 15s 118ms/step - loss: 0.5606 - accuracy: 0.3080
Epoch 4/300
125/125 [==============================] - 14s 114ms/step - loss: 0.5548 - accuracy: 0.2974
Epoch 5/300
125/125 [==============================] - 14s 113ms/step - loss: 0.5475 - accuracy: 0.2742
Epoch 6/300
125/125 [==============================] - 14s 116ms/step - loss: 0.5406 - accuracy: 0.2984
Epoch 7/300
125/125 [==============================] - 15s 119ms/step - loss: 0.5322 - accuracy: 0.3798
Epoch 8/300
125/125 [==============================] - 14s 115ms/step - loss: 0.5280 - accuracy: 0.3770
Epoch 9/300
125/125 [==============================] - 15s 119ms/step - loss: 0.5248 - accuracy: 0.3972
Epoch 10/300
125/125 [==============================] - 15s 116ms/step - los